In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
#import tushare as ts
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from torch.utils.data import TensorDataset
from tqdm import tqdm
import torch.nn.functional as F
import scipy.io as scio
import time 

In [2]:
path_noisy = r'Input_Patches_3Dreal2.csv'

In [3]:
data = pd.read_csv(path_noisy, header=None)

In [4]:
#type(data.shape[1])

In [5]:
data = data.T

In [6]:
data

,0,1,2,3,4,5,6,7,8,9,...,1718,1719,1720,1721,1722,1723,1724,1725,1726,1727
0,-1.815578,0.763338,0.754059,-1.111491,1.265898,2.840581,-0.011243,-1.028299,-0.803838,1.447028,...,-0.633431,-0.186204,-0.370927,0.299487,0.144882,0.070218,0.506113,0.159219,0.114936,0.496620
1,0.563465,0.617968,0.211230,0.996051,0.504243,0.208796,-0.138585,-0.537625,-0.622966,-0.595773,...,0.337196,0.079985,-0.198906,-0.382787,0.023340,-0.488307,-0.297148,-0.237758,-0.083351,0.332818
2,0.000380,-0.282182,-0.685111,-0.913809,-0.829509,-0.605626,-0.486916,-0.001499,0.206933,0.446442,...,-0.299653,-0.937786,-0.769336,-0.705439,-0.713293,-0.668061,-1.134946,-0.967494,-0.506291,-0.188805
3,0.320812,0.440913,-0.291419,-1.192542,-0.550346,-0.944047,-0.571363,0.524471,0.946202,0.685451,...,-1.379518,-0.845387,-2.753046,-3.564230,0.005390,-1.268847,-1.281567,-1.255031,-0.646878,3.104398
4,0.964060,0.934025,0.380661,0.482202,0.363099,0.522866,0.348136,0.118047,0.602362,0.407447,...,-0.206029,-0.272214,-0.910142,-0.683648,-0.899608,-0.422113,-0.247000,0.337039,0.876563,0.462564
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12579,-0.271112,0.465888,0.529884,0.787988,0.880660,0.726869,0.886427,0.810713,0.832469,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
12580,0.013781,0.578822,0.341261,0.726888,1.082105,0.865496,0.875879,0.631425,0.361306,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
12581,0.176536,-0.336254,-0.029643,0.222905,0.506781,0.452772,1.197900,1.282751,0.930060,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
12582,0.588327,1.053636,1.304401,1.647281,-0.391069,-0.183096,-1.315025,1.326614,1.962560,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [7]:
data = data.astype(np.float32)
data= torch.from_numpy(data.values)

In [8]:
data.shape

torch.Size([12584, 1728])

In [9]:
#将前80%作为训练集，后20%作为测试集
train_size = int(len(data) * 0.8)
train = data[:train_size]
vaild = data[train_size:]
print(train.shape)
print(vaild.shape)
batch_size1 = 64
w1 = 12
w2 = 12
w3 = 12

train_data= TensorDataset(train)
vaild_data= TensorDataset(vaild)

train_loader = torch.utils.data.DataLoader(train_data,
                                           batch_size = 64,
                                           shuffle = True)

vaild_loader = torch.utils.data.DataLoader(vaild_data,
                                          batch_size = 64,
                                          shuffle = True)

torch.Size([10067, 1728])
torch.Size([2517, 1728])


In [10]:
#定义Fully connected (FC) block
class FCB(nn.Module):
    def __init__(self, input_size, output_size, dropout=0.1):
        super().__init__()
        
        self.fc = nn.Linear(input_size, output_size)
        self.activation = nn.LeakyReLU()
        self.bn = nn.BatchNorm1d(output_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.fc(x)
        x = self.activation(x) 
        x = self.bn(x)
        x = self.dropout(x)
        
        return x

In [11]:
class Encoder(nn.Module):
    def __init__(self, input_size, dropout=0.1):
        super().__init__()


        self.fcb0 = FCB(input_size, 128, dropout)
        self.fcb1 = FCB(128, 64, dropout)

        self.fcb2 = FCB(64, 32, dropout)

        self.fcb3 = FCB(32, 16, dropout)

        self.fcb4 = FCB(16, 8, dropout)

        self.fcb5 = FCB(8, 4, dropout)

        self.fcb6 = FCB(4, 4, dropout)

        


    def forward(self, x):

        #x = x.reshape(x.shape[0],1,x.shape[1])

        x1 = self.fcb0(x)#x->128
        x2 = self.fcb1(x1)

        x3 = self.fcb2(x2)

        x4 = self.fcb3(x3)

        x5 = self.fcb4(x4)

        x6 = self.fcb5(x5)

        x7 = self.fcb6(x6)

        x8 = self.fcb6(x7)
        

        

        return x2,x3,x4,x5,x6,x7,x8

In [12]:
class Decoder(nn.Module):
    def __init__(self, output_size, dropout=0.1):
        super().__init__()
        self.pab1 = FCB(8, 8, dropout)
        self.pab2 = FCB(16, 16, dropout)
        self.pab3 = FCB(32, 32, dropout)
        self.pab4 = FCB(64, 64, dropout)
        self.pab5 = FCB(128, 128, dropout)
        self.pab6 = FCB(128, output_size, dropout)

        self.activation = nn.Identity()


    def forward(self,x2, x3,x4,x5,x6,x7,x8):
        
        x = torch.cat((x6,x8),dim=1)
        x = self.pab1(x)
        x = torch.cat((x,x5),dim=1)
        x = self.pab2(x)
        x = torch.cat((x,x4),dim=1)
        x = self.pab3(x)
        x = torch.cat((x,x3),dim=1)
        x = self.pab4(x)
        x = torch.cat((x,x2),dim=1)
        x = self.pab5(x)
        x = self.pab6(x)

        x = self.activation(x)
        return x

In [13]:
class AutoEncoder(nn.Module):
    def __init__(self,input_size, output_size):
        super().__init__()
        self.encoder = Encoder(input_size)
        self.decoder = Decoder(output_size)

    def forward(self, x):
        x2, x3,x4,x5,x6,x7,x8= self.encoder(x)
        x = self.decoder(x2, x3,x4,x5,x6,x7,x8)

        return x

In [14]:
device = torch.device("cuda") 
model = AutoEncoder(w1*w2*w3,w1*w2*w3).to(device)
#将模型转移到GPU上
#criterion = MeanHuberLoss(delta=1.3)
#criterion = WelschLoss(delta=0.5)
#criterion = Loss0(delta=0.46,r=0.05)#0.5 and 0.2,SNR:-8dB ok
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr = 0.001)

In [15]:
start_time = time.time()
es_cnt = 0
es_thres = 5
prev_train_loss = float('inf')
loss_train = []
loss_vaild = []
num_epochs = 100 # 总训练轮数
#num_batch_train = 0
for epoch in range(num_epochs):
  #train_bar = tqdm(train_loader)
  train_loss = 0.0
  
  for i , (batch) in enumerate(train_loader):

    # 数据转到device
    train_batch = batch[0].to(device)
    
    # 训练步骤  
    optimizer.zero_grad()
    outputs = model(train_batch)
    loss = criterion(outputs, train_batch)
    loss.backward() 
    optimizer.step()
    
    train_loss += loss.item()
    #num_batch_train +=1
  #train_loss除以所有bacth个数
  train_loss = train_loss/(np.ceil(train.size(0)/batch_size1))
  loss_train.append(train_loss)
    



  # 验证
  valid_loss = 0.0
  #num_batch_vaild = 0
  with torch.no_grad():
    for i , (batch) in enumerate(vaild_loader):
    #for batch in vaild_loader:
    
      val_batch = batch[0].to(device)
      
      outputs = model(val_batch)
      loss = criterion(outputs, val_batch)
      valid_loss += loss.item()
      #num_batch_vaild += 1
    valid_loss = valid_loss/(np.ceil(vaild.size(0)/batch_size1))
    loss_vaild.append(valid_loss)
    print("Epoch [{}/{}], Train Loss: {:.4f}, Valid Loss: {:.4f}".format(epoch+1, num_epochs, train_loss, valid_loss))

    
    # Early stopping
    if train_loss - prev_train_loss >= 0:
        es_cnt += 1
    else:
        #es_cnt = 0
        pass

    if es_cnt >= es_thres:
        print(f"Early stopped at epoch {epoch}, train loss stop improving")
        break  

    prev_train_loss = train_loss
  print('loss_train: ', loss_train)
  print('vaild_train: ',loss_vaild)          
print("Training finished")
current_time = time.time()
time_sum = current_time-start_time
print(time_sum)

Epoch [1/100], Train Loss: 0.9191, Valid Loss: 0.6239
loss_train:  [0.9191135364242747]
vaild_train:  [0.6238858565688133]
Epoch [2/100], Train Loss: 0.5371, Valid Loss: 0.4338
loss_train:  [0.9191135364242747, 0.5370767999298965]
vaild_train:  [0.6238858565688133, 0.43380031511187556]
Epoch [3/100], Train Loss: 0.4009, Valid Loss: 0.3491
loss_train:  [0.9191135364242747, 0.5370767999298965, 0.4009127979037128]
vaild_train:  [0.6238858565688133, 0.43380031511187556, 0.34909494817256925]
Epoch [4/100], Train Loss: 0.3376, Valid Loss: 0.3102
loss_train:  [0.9191135364242747, 0.5370767999298965, 0.4009127979037128, 0.33762622634066813]
vaild_train:  [0.6238858565688133, 0.43380031511187556, 0.34909494817256925, 0.3101947419345379]
Epoch [5/100], Train Loss: 0.3054, Valid Loss: 0.2911
loss_train:  [0.9191135364242747, 0.5370767999298965, 0.4009127979037128, 0.33762622634066813, 0.30542705877672266]
vaild_train:  [0.6238858565688133, 0.43380031511187556, 0.34909494817256925, 0.3101947419345

In [16]:
loss_train = pd.DataFrame(loss_train)
loss_vaild = pd.DataFrame(loss_vaild)

loss = pd.concat([loss_train,loss_vaild],axis=1)

In [17]:
loss.columns = ['train_loss','vaild_loss']

In [18]:
torch.save(model.state_dict(), r'.\model_3dreal2patch.pth')

In [19]:
model = AutoEncoder(w1*w2*w3,w1*w2*w3).to(device)
data = data.to(device)
model.load_state_dict(torch.load(r'.\model_3dreal2patch.pth'))
model.eval()
with torch.no_grad():
    output = model(data)
    #loss = criterion(output, data)
print(output.shape)

torch.Size([12584, 1728])


In [20]:
output = output.cpu()
output = output.numpy()
output = pd.DataFrame(output)

In [21]:
loss.to_csv(r'loss_3dreal2patch.csv',index=False)

In [22]:
output.to_csv(r'./output_3dreal2patch.csv',index=None,header=None)

In [23]:
#loss_0 = []
#start_time = time.time()
#es_cnt = 0
#es_thres = 5
#prev_train_loss = float('inf')
#loss_train = []
#loss_vaild = []
#num_epochs = 100 # 总训练轮数
##num_batch_train = 0
#for epoch in range(num_epochs):
#  #train_bar = tqdm(train_loader)
#  train_loss = 0.0
#  
#  for i , (batch) in enumerate(train_loader):
#
#    # 数据转到device
#    train_batch = batch[0].to(device)
#    
#    # 训练步骤  
#    optimizer.zero_grad()
#    outputs = model(train_batch)
#    loss = criterion(outputs, train_batch)
#    loss.backward() 
#    optimizer.step()
#
#    loss_0.append(loss.item())
#
#    train_loss += loss.item()
#    #num_batch_train +=1
#  #train_loss除以所有bacth个数
#  train_loss = train_loss/(np.ceil(train.size(0)/batch_size1))
#  loss_train.append(train_loss)
#    
#
#
#
#  # 验证
#  valid_loss = 0.0
#  #num_batch_vaild = 0
#  with torch.no_grad():
#    for i , (batch) in enumerate(vaild_loader):
#    #for batch in vaild_loader:
#    
#      val_batch = batch[0].to(device)
#      
#      outputs = model(val_batch)
#      loss = criterion(outputs, val_batch)
#      valid_loss += loss.item()
#      #num_batch_vaild += 1
#    valid_loss = valid_loss/(np.ceil(vaild.size(0)/batch_size1))
#    loss_vaild.append(valid_loss)
#    print("Epoch [{}/{}], Train Loss: {:.4f}, Valid Loss: {:.4f}".format(epoch+1, num_epochs, train_loss, valid_loss))
#
#    
#    # Early stopping
#    if train_loss - prev_train_loss >= 0:
#        es_cnt += 1
#    else:
#        #es_cnt = 0
#        pass
#
#    if es_cnt >= es_thres:
#        print(f"Early stopped at epoch {epoch}, train loss stop improving")
#        break  
#    
#
#
##   if epoch == 10:
##     torch.save(model.state_dict(), r'.\model_epochs10.pth')
##   elif epoch == 20:
##     torch.save(model.state_dict(), r'.\model_epochs20.pth')
##   elif epoch == 30:
##     torch.save(model.state_dict(), r'.\model_epochs30.pth')
##   elif epoch == 40:
##     torch.save(model.state_dict(), r'.\model_epochs40.pth')
##   elif epoch == 50:
##     torch.save(model.state_dict(), r'.\model_epochs50.pth')
##   elif epoch == 60:
##     torch.save(model.state_dict(), r'.\model_epochs60.pth')
##   elif epoch == 70:
##     torch.save(model.state_dict(), r'.\model_epochs70.pth')
##   elif epoch == 80:
##     torch.save(model.state_dict(), r'.\model_epochs80.pth')
##   elif epoch == 90:
##     torch.save(model.state_dict(), r'.\model_epochs90.pth')
##   elif epoch == 100:
##     torch.save(model.state_dict(), r'.\model_epochs100.pth')
##   else:
##       pass
#
#    prev_train_loss = train_loss
#  print('loss_train: ', loss_train)
#  print('vaild_train: ',loss_vaild)          
#print("Training finished")
#current_time = time.time()
#time_sum = current_time-start_time
#print(time_sum)